In [1]:
import pandas as pd
import geopandas as gpd
import os

In [2]:
hospital_bed = pd.read_stata("../covid_data/hospitals/dlhs4_hospitals_dist.dta")

hospital_bed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lgd_state_id            636 non-null    object 
 1   lgd_district_id         636 non-null    object 
 2   dlhs4_dh_beds           636 non-null    float64
 3   dlhs4_dh_count          636 non-null    float64
 4   dlhs4_dh_staff          636 non-null    float64
 5   dlhs4_chc_beds          636 non-null    float64
 6   dlhs4_chc_count         636 non-null    float64
 7   dlhs4_chc_staff         636 non-null    float64
 8   dlhs4_phc_beds          636 non-null    float64
 9   dlhs4_phc_count         636 non-null    float64
 10  dlhs4_phc_staff         636 non-null    float64
 11  dlhs4_phc_pop           636 non-null    float64
 12  pc11_pca_tot_p          636 non-null    float64
 13  dlhs4_phc_mult          636 non-null    float32
 14  dlhs4_total_beds        636 non-null    fl

In [3]:
hospital_bed_state = hospital_bed.groupby("lgd_state_id")[["dlhs4_dh_beds", "pc11_pca_tot_p"]].sum().reset_index()

hospital_bed_state

,lgd_state_id,dlhs4_dh_beds,pc11_pca_tot_p
0,02,2330.000000,6.864602e+06
1,03,5119.000000,2.774334e+07
2,04,550.000000,1.055450e+06
3,05,4360.000000,1.008629e+07
4,06,3965.000000,2.535146e+07
5,08,10445.000000,6.768059e+07
6,09,17651.999997,1.968109e+08
7,10,6291.000000,1.033986e+08
8,11,504.000000,6.105770e+05
9,12,1050.000003,1.383727e+06


In [4]:
hospital_bed_state["bed_1000"] = (hospital_bed_state["dlhs4_dh_beds"] / hospital_bed_state["pc11_pca_tot_p"]) * 1000

hospital_bed_state

,lgd_state_id,dlhs4_dh_beds,pc11_pca_tot_p,bed_1000
0,02,2330.000000,6.864602e+06,0.339422
1,03,5119.000000,2.774334e+07,0.184513
2,04,550.000000,1.055450e+06,0.521105
3,05,4360.000000,1.008629e+07,0.432270
4,06,3965.000000,2.535146e+07,0.156401
5,08,10445.000000,6.768059e+07,0.154328
6,09,17651.999997,1.968109e+08,0.089690
7,10,6291.000000,1.033986e+08,0.060842
8,11,504.000000,6.105770e+05,0.825449
9,12,1050.000003,1.383727e+06,0.758820


In [5]:
state_shp = gpd.read_file("./output_shpfile/state_key_with_polygon/state_key_with_polygon.shp")

state_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   lgd_sta_id  36 non-null     object  
 1   lgd_sta_na  36 non-null     object  
 2   pc11_st_id  36 non-null     object  
 3   pc11_st_na  36 non-null     object  
 4   geometry    36 non-null     geometry
dtypes: geometry(1), object(4)
memory usage: 1.5+ KB


In [6]:
hospital_bed_state_shp = pd.merge(state_shp, hospital_bed_state, left_on="lgd_sta_id", right_on="lgd_state_id", how="left")

hospital_bed_state_shp = hospital_bed_state_shp[hospital_bed_state_shp["lgd_sta_na"].isin(["tamil nadu", "madhya pradesh", "maharashtra", "uttar pradesh", "kerala", "bihar"])]

hospital_bed_state_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 6 entries, 7 to 31
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   lgd_sta_id      6 non-null      object  
 1   lgd_sta_na      6 non-null      object  
 2   pc11_st_id      6 non-null      object  
 3   pc11_st_na      6 non-null      object  
 4   geometry        6 non-null      geometry
 5   lgd_state_id    6 non-null      object  
 6   dlhs4_dh_beds   6 non-null      float64 
 7   pc11_pca_tot_p  6 non-null      float64 
 8   bed_1000        6 non-null      float64 
dtypes: float64(3), geometry(1), object(5)
memory usage: 480.0+ bytes


In [7]:
hospital_bed_state_shp.drop(columns=["lgd_state_id", "pc11_pca_tot_p"], inplace=True)
hospital_bed_state_shp.rename(columns={"dlhs4_dh_beds": "hospit_bed"}, inplace=True)

hospital_bed_state_shp.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 6 entries, 7 to 31
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   lgd_sta_id  6 non-null      object  
 1   lgd_sta_na  6 non-null      object  
 2   pc11_st_id  6 non-null      object  
 3   pc11_st_na  6 non-null      object  
 4   geometry    6 non-null      geometry
 5   hospit_bed  6 non-null      float64 
 6   bed_1000    6 non-null      float64 
dtypes: float64(2), geometry(1), object(4)
memory usage: 384.0+ bytes


In [8]:
# Export the "mortality_perDistrict_perMonth_shp" table to shapefile
os.mkdir("./output_shpfile/hospital_bed_state_shp")
hospital_bed_state_shp.to_file("./output_shpfile/hospital_bed_state_shp/hospital_bed_state_shp.shp", driver="ESRI Shapefile")